# Using the Weather Analysis Tools with Claude API
## An interactive demo combining NWS API data with Claude's analytical capabilities

![Weather Analysis Tool](../img/gradio_ui.png)

### Table of Contents

- [Installation](#installation)
- [Setup](#setup)
- [Weather API Functions](#weather-api-functions)
- [Claude Integration](#claude-integration)
- [Example Usage](#example-usage)
- [Gradio Web Interface](#gradio-web-interface)
    - [Weather Forecast Analysis](#weather-forecast-analysis-with-claude--nws)
    - [Weather Alerts](#weather-alerts-with-claude--nws)
- [Terminate the Demo](#terminate-the-demo)

### Installation
#### First, install the necessary packages:

In [ ]:
# First, install the necessary packages:
%pip install httpx nest_asyncio anthropic ipywidgets gradio

### Setup
#### Import required libraries and set up the environment:

In [ ]:
# Load API keys from secrets.json

import os
import json
import sys

def get_secrets():
    with open('secrets.json') as secrets_file:
        secrets = json.load(secrets_file)

    return secrets

if __name__ == "__main__":
    secrets = get_secrets()
    os.environ["ANTHROPIC_API_KEY"]  = secrets.get("ANTHROPIC_API_KEY") # Add or Replace in secrets.json file
    os.environ["LAYER_DEMO_AUTH_CLIENT_SECRET"]  = secrets.get("LAYER_DEMO_AUTH_CLIENT_SECRET")


    # Dynamically add the project root directory to PYTHONPATH
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    sys.path.append(project_root)
    # Add the helper directory to the Python path
    current_dir = os.path.dirname(os.path.abspath('__file__'))
    helper_dir = os.path.join(current_dir, 'helper')
    sys.path.append(helper_dir)
    # Verify the project root is in the Python path
    print("Project root added to PYTHONPATH:", project_root)


In [ ]:
import os
from layer_sdk_integration.layer_sdk import OIDCClientCredentials, layer

keycloak_base_path = ""
realm_name = ""
client_id = ""
client_secret = os.getenv("LAYER_DEMO_AUTH_CLIENT_SECRET")  # Ensure the client secret is set
auth_provider = OIDCClientCredentials(
    token_url=f"https://{keycloak_base_path}/realms/{realm_name}/protocol/openid-connect/token",
    client_id=client_id,
    client_secret=client_secret,
)

application_id = "" 

layer.init(
    base_url="",
    application_id=application_id,
    environment="sandbox",
    auth_provider=auth_provider,
    firewall_base_url="",
    enable_firewall_instrumentation=True
)

In [ ]:
# Test Layer SDK initialization
try:
    session_id = layer.create_session(attributes={"user.id": "ari@protectai.com"})
    print(f"Test session created: {session_id}")
except Exception as e:
    print(f"Error testing Layer SDK: {e}")

In [ ]:
import json
import os
import httpx
import asyncio
from typing import Any, Dict, List
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
import anthropic
import nest_asyncio
import gradio
from weather_tools import log_as_tool
from datetime import datetime, timezone

# Apply nest_asyncio to fix event loop issues in Jupyter
nest_asyncio.apply()

### Weather API Functions
#### Functions to interact with the National Weather Service API:

In [ ]:
# Constants for weather API
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"

# Weather API functions
async def make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Error making request to {url}: {e}")
            return None

@log_as_tool(layer)
async def get_alerts_async(state: str) -> dict:
    """Get weather alerts for a US state and return structured data."""
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)
    
    if not data or "features" not in data:
        return {"success": False, "message": "Unable to fetch alerts or no alerts found."}
    
    if not data["features"]:
        return {"success": True, "alerts": [], "message": "No active alerts for this state."}
    
    return {"success": True, "api_url": url, "alerts": data["features"], "raw_data": data}

@log_as_tool(layer)
async def get_forecast_async(latitude: float, longitude: float) -> dict:
    """Get weather forecast for a location and return structured data."""
    # First get the forecast grid endpoint
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = await make_nws_request(points_url)

    if not points_data:
        return {"success": False, "message": "Unable to fetch forecast data for this location."}

    # Get location information
    location_info = {
        "city": points_data["properties"].get("relativeLocation", {}).get("properties", {}).get("city", "Unknown"),
        "state": points_data["properties"].get("relativeLocation", {}).get("properties", {}).get("state", "Unknown"),
        "coordinates": f"{latitude}, {longitude}"
    }

    # Get the forecast URL from the points response
    forecast_url = points_data["properties"]["forecast"]
    forecast_data = await make_nws_request(forecast_url)

    if not forecast_data:
        return {"success": False, "message": "Unable to fetch detailed forecast."}

    # Get the forecast periods
    periods = forecast_data["properties"]["periods"]
    
    return {
        "success": True, 
        "api_url": points_url,
        "location": location_info,
        "periods": periods,
        "raw_data": forecast_data
    }

def log_tool(func):
    def wrapper(*args, **kwargs):
        # Execute function first to ensure it works
        start_time = datetime.now(timezone.utc)
        result = func(*args, **kwargs)
        end_time = datetime.now(timezone.utc)
        if isinstance(result, tuple) and len(result) == 2:
            response = result[0]  # The actual analysis
            api_url = result[1]  # The API URL
        try:
            layer.append_action(
                session_id=session_id,
                kind="completion_output",
                start_time=start_time,
                end_time=end_time,
                attributes={"tool.api": api_url, "tool.name": func.__name__, "tool.arguments": str(kwargs)},
                data={"messages": [{"content": str(response)[:100], "role": "assistant"}]}
            )
            print(f"Logged action for {func.__name__}")
        except Exception as e:
            print(f"Error: {e}")
            
        return result
    return wrapper


### Claude Integration

#### Initialize Anthropic Client

In [ ]:
# Load credentials from secrets.json
def load_credentials():
    """Load Anthropic API key from secrets.json file."""
    try:
        return secrets.get("ANTHROPIC_API_KEY")
    except Exception as e:
        print(f"Error loading credentials: {e}")
        print(f"Please ensure the file exists at: {secrets_file}")
        print("And that it contains an 'ANTHROPIC_API_KEY' field")
        return None

api_key = load_credentials()
claude_client = anthropic.Anthropic(api_key=api_key)

#### Function to analyze weather data using Claude:

In [ ]:
# Claude integration functions
def analyze_weather_with_claude(weather_data: dict, analysis_type: str = "general") -> str:
    """
    Send weather data to Claude and get back an analysis.
    
    Args:
        weather_data: Weather data from the NWS API
        analysis_type: Type of analysis to request from Claude
    
    Returns:
        Claude's analysis as a string
    """
    if not api_key:
        return "⚠️ ERROR: Anthropic API key not found in secrets.json file."
        
    # Create a prompt based on the type of analysis requested
    prompt_prefix = ""
    
    if analysis_type == "general":
        prompt_prefix = "Please analyze this weather data and provide a clear summary of the key points. Include any notable patterns or concerns:"
    elif analysis_type == "travel":
        prompt_prefix = "Based on this weather data, provide travel recommendations. How might this weather impact travel plans? Are there any safety concerns or precautions travelers should take?"
    elif analysis_type == "emergency":
        prompt_prefix = "Analyze this weather data for potential emergency situations. Provide clear guidance on safety precautions people should take. Highlight any severe threats and recommended actions:"
    # elif analysis_type == "agricultural":
    #     prompt_prefix = "Analyze this weather forecast from an agricultural perspective. What impacts might this weather have on farming operations, crops, or livestock? What recommendations would you give to farmers?"
    
    # Convert weather data to a string for the prompt
    weather_json = json.dumps(weather_data, indent=2)
    
    # Construct the full prompt
    full_prompt = f"{prompt_prefix}\n\nWeather Data:\n```json\n{weather_json}\n```"
    
    # Call Claude API
    try:
        message = claude_client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=1000,
            temperature=0.3,
            system="You are a weather analysis assistant. Provide clear, concise analysis of weather data. Focus on practical implications and actionable insights. Use bullet points where appropriate for clarity.",
            messages=[
                {"role": "user", "content": full_prompt}
            ]
        )
        
        # Return Claude's response
        return message.content[0].text
    except Exception as e:
        return f"⚠️ Error calling Claude API: {str(e)}"

### Example Usage
#### Example of using the weather analysis directly:

In [ ]:
# Example: Using analyze_weather_with_claude directly

# First make sure you've imported all the necessary functions and initialized the Claude client
# This assumes you've already run the main code we developed earlier

# Let's get a forecast for Austin and analyze it directly
def analyze_weather(session_id=None):
    # Get the forecast data
    forecast = asyncio.run(get_forecast_async(30.2672, -97.7431))
    
    if not forecast["success"]:
        print(f"Error: {forecast['message']}")
        return "Error retrieving forecast data"
    
    # Print some basic information about the forecast
    location = forecast["location"]
    print(f"Got forecast for {location['city']}, {location['state']}")
    print(f"Number of forecast periods: {len(forecast['periods'])}")
    
    # Let's analyze it from different perspectives
    analysis_type = "general"  # Start with general analysis
    
    # Call the analyze function directly
    analysis = analyze_weather_with_claude(forecast, analysis_type)
    # Extract the API URL for logging
    api_url = forecast.get("api_url", "unknown")

    return analysis, api_url

# Apply the log_tool decorator to the synchronous function
analyze_weather = log_tool(analyze_weather)

# Now call it - it should return the actual text
result = analyze_weather(session_id=session_id)
print(result)

### Gradio Interface
#### Create an interactive web interface using Gradio that calls`analyze_weather()`:
1. No Layer session is created when the app first loads
2. A session is only created when a user interacts with the chatbot
3. All subsequent actions should be logged to the same session

In [ ]:
import gradio as gr
import nest_asyncio
from datetime import datetime, timezone
from layer_sdk_integration.layer_sdk import layer
import json
import asyncio

# Apply nest_asyncio to fix event loop issues in Jupyter
nest_asyncio.apply()

# Initialize session_id as None (no session at startup)
session_id = None

# Function to get or create a Layer session
def get_or_create_session():
    global session_id
    
    # If we already have a session, return it
    if session_id:
        print(f"Using existing session: {session_id}")
        return session_id
    
    # Otherwise create a new session
    try:
        new_session_id = layer.create_session(attributes={"user.id": "ari@protectai.com"})
        session_id = new_session_id
        print(f"Created new Layer session: {session_id}")
        return session_id
    except Exception as e:
        print(f"Error creating Layer session: {e}")
        # Fallback to a default session ID if Layer session creation fails
        session_id = None
        return session_id

# The main function that analyzes weather and logs to Layer
def analyze_weather(latitude=30.2672, longitude=-97.7431, analysis_type="general"):
    # Ensure we have a session
    current_session = get_or_create_session()
    print(f"Starting analysis with session_id: {current_session}")
    
    # Get the forecast data
    forecast = asyncio.run(get_forecast_async(latitude, longitude))
    
    if not forecast["success"]:
        print(f"Error: {forecast['message']}")
        return "Error retrieving forecast data"
    
    # Print some basic information about the forecast
    location = forecast["location"]
    print(f"Got forecast for {location['city']}, {location['state']}")
    print(f"Number of forecast periods: {len(forecast['periods'])}")
    
    # Call the analyze function with the specified analysis type
    analysis = analyze_weather_with_claude(forecast, analysis_type)
    
    # Extract the API URL for logging
    api_url = forecast.get("api_url", "unknown")

    # Log this action to Layer
    try:
        start_time = datetime.now(timezone.utc)
        end_time = datetime.now(timezone.utc)
        
        layer.append_action(
            session_id=current_session,
            kind="completion_output",
            start_time=start_time,
            end_time=end_time,
            attributes={
                "tool.api": api_url,
                "tool.name": "get_forecast_async",
                "tool.arguments": f"latitude={latitude}, longitude={longitude}, analysis_type={analysis_type}"
            },
            data={"messages": [{"content": analysis[:100], "role": "assistant"}]}
        )
        print(f"Successfully logged action to Layer")
    except Exception as e:
        print(f"Error logging to Layer: {e}")
    
    return analysis

# Function to run the alerts check
def run_alerts_check(state):
    current_session = get_or_create_session()
    print(f"Starting analysis with session_id: {current_session}")
    
    # Get the alerts data
    alerts = asyncio.run(get_alerts_async(state))
    
    if not alerts["success"]:
        print(f"Error: {alerts.get('message', 'Unknown error')}")
        return "Error retrieving alerts data"
    
    # If there are no alerts, return a simple message
    if not alerts.get("alerts"):
        return f"Good news! There are currently no active weather alerts for {state}."
    
    # Otherwise, analyze the alerts with Claude
    analysis = analyze_weather_with_claude(alerts, "emergency")
    # Extract the API URL for logging
    api_url = alerts.get("api_url", "unknown")
        # Log this action to Layer
    try:
        start_time = datetime.now(timezone.utc)
        end_time = datetime.now(timezone.utc)
        
        layer.append_action(
            session_id=current_session,
            kind="completion_output",
            start_time=start_time,
            end_time=end_time,
            attributes={
                "tool.api": api_url,
                "tool.name": "get_alerts_async",
                "tool.arguments": f"state={state}, analysis_type='emergency'"
            },
            data={"messages": [{"content": analysis[:100], "role": "assistant"}]}
        )
        print(f"Successfully logged action to Layer")
    except Exception as e:
        print(f"Error logging to Layer: {e}")
        
    return analysis

# Create a function to display the current session status
def get_session_status():
    if session_id:
        return f"**Session ID:** {session_id}"
    else:
        return "**Session Status:** No active session"

# Create the Gradio interface
with gr.Blocks(title="Weather Analysis with Claude") as demo:
    gr.Markdown("# 🌦️ Weather Analysis with Claude")
    gr.Markdown("Enter coordinates and select the type of analysis you want Claude to perform.")
    
    # Display session status (will be updated)
    session_status = gr.Markdown(get_session_status())
    
    with gr.Row():
        # Left column for inputs and examples
        with gr.Column(scale=1):
            latitude = gr.Number(label="Latitude", value=30.2672)
            longitude = gr.Number(label="Longitude", value=-97.7431)
            analysis_type = gr.Radio(
                ["general", "travel", "emergency"],
                label="Analysis Type",
                value="general"
            )
            analyze_button = gr.Button("Analyze Weather", variant="primary")
            
            # Add examples section in the same column using a Group
            with gr.Group():
                gr.Markdown("### Example Coordinates")
                gr.Markdown("- Austin, TX: 30.2672, -97.7431")
                gr.Markdown("- New York, NY: 40.7128, -74.0060")
                gr.Markdown("- San Francisco, CA: 37.7749, -122.4194")
                gr.Markdown("- Chicago, IL: 41.8781, -87.6298")
            
                    # Alert inputs
            with gr.Group():
                gr.Markdown("### Weather Alerts")
                state = gr.Text(label="State Code (e.g., TX, CA, NY)", value="TX")
                alerts_button = gr.Button("Get Alerts", variant="stop")
        
        # Right column for output
        with gr.Column(scale=2):
            # Add a status indicator
            status = gr.Markdown("Ready for analysis")
            output = gr.Markdown()
    
    # Update the click event to show thinking status and track analysis
    analyze_button.click(
        # First callback: update status to "Thinking..."
        fn=lambda: "### 🔄 Analyzing weather data and generating insights...\n\nPlease wait while Claude processes your request.",
        inputs=None,
        outputs=status
    ).then(
        # Second callback: perform the actual analysis
        fn=analyze_weather,
        inputs=[latitude, longitude, analysis_type],
        outputs=output
    ).then(
        # Third callback: update status when done and refresh session display
        fn=lambda: ["### ✅ Analysis complete!", get_session_status()],
        inputs=None,
        outputs=[status, session_status]
    )
    # Update the alerts button click event
    alerts_button.click(
        # First callback: update status
        fn=lambda: "### 🔄 Checking weather alerts...\n\nPlease wait while we retrieve and analyze active alerts.",
        inputs=None,
        outputs=status
    ).then(
        # Second callback: perform alerts check
        fn=run_alerts_check,
        inputs=[state],
        outputs=output
    ).then(
        # Third callback: update status and session display
        fn=lambda: ["### ✅ Alert check complete!", get_session_status()],
        inputs=None,
        outputs=[status, session_status]
    )
# Launch the app in a browser (not embedded in the notebook)
demo.launch(inline=False, share=True)

### Weather Forecast Analysis with Claude & NWS

![Weather Analysis with Claude](../img/forecast.png)

### Weather Alerts with Claude & NWS
![Weather Analysis with Claude](../img/alerts.png)

### Terminate the Demo

In [ ]:
demo.close()